# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
# Create aiport dataframe
clean_city_data = pd.read_csv('../output_data/city_data.csv')
weather_df.dropna()
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.60,168.33,37.00,93,54,3.00,NZ,1591993398
1,1,upernavik,72.79,-56.15,32.67,94,0,3.11,GL,1591993408
2,2,hamilton,39.18,-84.53,84.99,28,40,6.93,US,1591993408
3,3,tasiilaq,65.61,-37.64,39.20,80,6,14.99,GL,1591993408
4,4,busselton,-33.65,115.33,48.99,81,68,10.58,AU,1591993408


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# gmaps
gmaps.configure(api_key=g_key)

In [33]:
locations = clean_city_data[["Lat", "Lng"]]
weights = clean_city_data[["Humidity"]]
# Fill NaN values and convert to float
weights= clean_city_data["Humidity"].astype(float)

In [34]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heatlayer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heatlayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
optimal_vacay = clean_city_data.copy()
optimal_vacay = optimal_vacay[optimal_vacay['Max Temp'] >= 80]
optimal_vacay = optimal_vacay[optimal_vacay['Max Temp'] <= 100]  
optimal_vacay = optimal_vacay[optimal_vacay['Wind Speed'] <= 10]
optimal_vacay = optimal_vacay[optimal_vacay['Cloudiness'] == 0]
optimal_vacay = optimal_vacay.dropna()
optimal_vacay = optimal_vacay.reset_index()
optimal_vacay

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,65,65,sunland park,38.89,-101.82,89.01,27,0,8.01,US,1591993585
1,92,92,crixas,-14.55,-49.97,84.56,47,0,3.56,BR,1591993588
2,155,155,filingue,14.35,3.32,90.41,48,0,9.69,NE,1591993597
3,207,207,riyadh,24.69,46.72,93.20,15,0,6.93,SA,1591993487
4,344,344,arraial do cabo,-22.84,-46.65,82.40,37,0,5.82,BR,1591993624
5,376,376,carlyle,38.61,-89.37,89.01,36,0,8.01,US,1591993635
6,395,395,baghdad,33.34,44.40,82.40,28,0,5.82,IQ,1591993653
7,436,436,goure,13.98,10.27,93.00,40,0,4.00,NE,1591993658
8,470,470,sawakin,19.11,37.33,88.14,43,0,5.95,SD,1591993663
9,516,516,awbari,26.59,12.78,94.01,11,0,5.99,LY,1591993669


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
#dropped unnamed column
hotel_df= optimal_vacay.drop(['Unnamed: 0'],axis=1)
#add Hotel Name Column
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,65,sunland park,38.89,-101.82,89.01,27,0,8.01,US,1591993585,
1,92,crixas,-14.55,-49.97,84.56,47,0,3.56,BR,1591993588,
2,155,filingue,14.35,3.32,90.41,48,0,9.69,NE,1591993597,
3,207,riyadh,24.69,46.72,93.20,15,0,6.93,SA,1591993487,
4,344,arraial do cabo,-22.84,-46.65,82.40,37,0,5.82,BR,1591993624,


In [43]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#xounter
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except 
    try:
        response = requests.get(base_url, params=params)
        hotel_names = response.json()
        hotels.append(hotel_names['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,65,sunland park,38.89,-101.82,89.01,27,0,8.01,US,1591993585,NaN
1,92,crixas,-14.55,-49.97,84.56,47,0,3.56,BR,1591993588,RISO HOTEL
2,155,filingue,14.35,3.32,90.41,48,0,9.69,NE,1591993597,NaN
3,207,riyadh,24.69,46.72,93.20,15,0,6.93,SA,1591993487,InterContinental Riyadh
4,344,arraial do cabo,-22.84,-46.65,82.40,37,0,5.82,BR,1591993624,Chácara Cantinho do Céu


In [44]:
# # find the closest restaurant of each type to coordinates
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#     target_coordinates = f"{row['Lat']},{row['Lng']}"
#     target_radius = 5000
#     target_type = "lodging"
#     params = {
#         "location": target_coordinates,
#         "radius": target_radius,
#         "type": target_type,
#         "key": g_key}
    
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     try:
#         response = requests.get(base_url, params=params).json()
#            # extract results
# #         results = response['results']
    
#     # get restaurant type from df
       


#     except (KeyError, IndexError):
#         print("NaN.")
        
# hotel_name = row['Hotel Name']
# hotel_df.loc[index, 'Hotel Name'] = results[0]['Hotel Name']
# hotel_df.head()
  

In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure

fig


Figure(layout=FigureLayout(height='420px'))